In [3]:
import pandas as pd
import regex as re
from unidecode import unidecode
from tqdm import tqdm


In [2]:
orbis = pd.read_csv("/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name_Combined/BvD_ID_and_Name_Full.csv")

In [2]:
ted = pd.read_csv("/Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/ted_all.csv")

/var/folders/r2/9vnxdxyd26380tms_8k538b40000gn/T/ipykernel_5312/1114671087.py:1: DtypeWarning: Columns (8,10,14,16,17,18,20,23,24,33,46,52,53,55,57,62) have mixed types. Specify dtype option on import or set low_memory=False.
  ted = pd.read_csv("/Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/ted_all.csv")


In [3]:
# Compile regex patterns once for efficiency
SUFFIXES_RE = re.compile(r'\b(inc|corp|ltd|llc|company|co|sas)\b', re.IGNORECASE)
SPECIAL_CHARS_RE = re.compile(r'[\p{P}\p{S}]', re.IGNORECASE)
MULTIPLE_SPACES_RE = re.compile(r'\s+')
NON_LATIN_RE = re.compile(r'[^\p{Latin}\s\d]', re.IGNORECASE)
NUMBERS_RE = re.compile(r'\d+')


def preprocess_name(name):
    """
    Preprocesses a company name by cleaning, transliterating, and extracting numbers.

    Parameters:
        name (str): The original company name.

    Returns:
        tuple: A tuple containing:
            - processed_name (str or None): The cleaned and transliterated name.
            - is_transliterated (bool): Flag indicating if the name was transliterated.
            - numbers (list): List of numbers extracted from the name.
    """
    if isinstance(name, float) or pd.isnull(name):  # Handle NaN or missing values
        return None, False, []

    # Convert to lowercase
    name = name.lower()

    # Extract numbers from the name
    numbers = NUMBERS_RE.findall(name)

    # Remove common company suffixes
    name_cleaned = SUFFIXES_RE.sub('', name)

    # Remove punctuation and symbols
    name_cleaned = SPECIAL_CHARS_RE.sub('', name_cleaned)

    # Normalize whitespace
    name_cleaned = MULTIPLE_SPACES_RE.sub(' ', name_cleaned).strip()

    # If the name is empty after cleaning, return None
    if not name_cleaned:
        return None, False, numbers

    # Check for non-Latin characters
    if NON_LATIN_RE.search(name_cleaned):
        # Transliterate non-Latin characters to Latin alphabet
        name_transliterated = unidecode(name_cleaned)
        return name_transliterated, True, numbers  # Return transliterated name, flag, and numbers
    else:
        return name_cleaned, False, numbers  # Return cleaned name, flag, and numbers

In [ ]:
orbis_clean = orbis.dropna(subset = ['bvdidnumber', 'name'])

In [4]:
ted_clean = ted.dropna(subset = ['WIN_NAME', 'WIN_COUNTRY_CODE'])

In [5]:
ted_clean.shape

(7892110, 75)

In [6]:
ted_clean[['processed_name', 'is_transliterated', 'numbers']] = ted_clean['WIN_NAME'].apply(
        lambda x: pd.Series(preprocess_name(x))
    )

/var/folders/r2/9vnxdxyd26380tms_8k538b40000gn/T/ipykernel_5312/1378794927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ted_clean[['processed_name', 'is_transliterated', 'numbers']] = ted_clean['WIN_NAME'].apply(
/var/folders/r2/9vnxdxyd26380tms_8k538b40000gn/T/ipykernel_5312/1378794927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ted_clean[['processed_name', 'is_transliterated', 'numbers']] = ted_clean['WIN_NAME'].apply(
/var/folders/r2/9vnxdxyd26380tms_8k538b40000gn/T/ipykernel_5312/13787949

In [9]:
ted_clean.to_csv('ted_transliterated.csv', index = False)

In [14]:
ted['WIN_COUNTRY_CODE']

0           IT
1           GR
2           HU
3           BE
4           BE
            ..
10496104    NO
10496105    NO
10496106    NO
10496107    NO
10496108    CH
Name: WIN_COUNTRY_CODE, Length: 10496109, dtype: object

In [15]:
# grab only the first two characters of each entry…
prefixes = ted['WIN_COUNTRY_CODE'].str[:2]

# …then get the unique values and their count
unique_prefixes = prefixes.unique()
print(unique_prefixes)
print("Count:", len(unique_prefixes))

['IT' 'GR' 'HU' 'BE' 'NL' 'FR' nan 'DE' 'AT' 'LT' 'UK' 'PL' 'LU' 'CZ' 'MC'
 'LV' 'SI' 'SK' 'HR' 'BG' 'RO' 'SE' 'ES' 'CH' 'EE' 'GP' 'CY' 'AL' 'US'
 'NO' 'PT' 'MT' 'IE' 'DK' 'FI' 'MK' 'RE' 'TR' 'IN' 'CA' 'BQ' 'AM' 'NC'
 'IS' 'DZ' 'GI' 'AU' 'JE' 'NP' 'BD' 'BY' 'KR' 'ZA' 'AF' 'FK' 'LI' 'MX'
 'CN' 'MQ' 'IL' 'IR' 'GF' '1A' 'ZW' 'HK' 'AD' 'ZM' 'RU' 'JP' 'PK' 'ID'
 'UG' 'UA' 'AE' 'FM' 'SM' 'RS' 'TL' 'CR' 'BR' 'SG' 'SV' 'GE' 'KE' 'NZ'
 'AZ' 'SA' 'GU' 'GT' 'TM' 'VI' 'LA' 'YT' 'MZ' 'GH' 'BH' 'MU' 'MY' 'GM'
 'AX' 'ET' 'UM' 'PA' 'SZ' 'LR' 'TC' 'PF' 'SD' 'SC' 'BJ' 'OM' 'AO' 'UY'
 'BO' 'TW' 'GL' 'SL' 'BA' 'LK' 'WF' 'MD' 'SR' 'CO' 'BI' 'NF' 'TZ' 'TN'
 'TH' 'DJ' 'VE' 'AI' 'IM' 'SY' 'ME' 'AW' 'PR' 'NE' 'FO' 'BM' 'KH' 'JO'
 'UZ' 'CD' 'LS' 'KZ' 'BS' 'ML' 'DO' 'BF' 'FJ' 'PM' 'MM' 'NR' 'EG' 'ER'
 'VN' 'CV' 'PH' 'KP' 'QA' 'CL' 'EC' 'JM' 'LB' 'KG' 'VG' 'DM' 'TO' 'RW'
 'SO' 'MA' 'CM' 'GS' 'MH' 'VU' 'PS' 'VA' 'MN' 'MF' 'SH' 'GA' 'AR' 'MW'
 'IO' 'AG' 'TT' 'BW' 'WS' 'TD' 'CW' 'BT' 'GG' 'BZ' 'IQ' 'NG' 'KW' 'NA'
 '

In [17]:
sum(ted['WIN_COUNTRY_CODE'] == 'CC')

3

In [18]:
# get the two‑letter prefixes
prefixes = ted['WIN_COUNTRY_CODE'].str[:2]

# count them
counts = prefixes.value_counts()

# select only those with >100 occurrences
common_codes = counts[counts > 100].index.tolist()

print(common_codes)

['PL', 'FR', 'DE', 'RO', 'UK', 'ES', 'IT', 'CZ', 'SI', 'BG', 'LT', 'SE', 'LV', 'NL', 'HU', 'BE', 'HR', 'DK', 'GR', 'FI', 'AT', 'NO', 'SK', 'PT', 'EE', 'IE', 'CH', 'MK', 'CY', 'LU', 'MT', 'US', 'IS', 'RE', 'GP', 'CA', 'MQ', 'CN', 'LI', 'RU', 'TR', 'IN', 'AW', 'AF', 'GF', 'IL', 'RS', 'JP', 'AM', 'AE', 'AU', 'KR', 'UA', 'ZA', 'SG', 'HK', 'SM', 'MC', 'YT', 'MD']


In [19]:
len(common_codes)

60

In [20]:
import os

# 1. compute two‑letter prefixes and filter common codes
prefixes = ted['WIN_COUNTRY_CODE'].str[:2]
counts = prefixes.value_counts()
common_codes = counts[counts > 100].index.tolist()

# 2. make output folder
output_dir = 'TED_by_relevant_country'
os.makedirs(output_dir, exist_ok=True)

# 3. for each code, subset and save
for code in common_codes:
    subset = ted[prefixes == code]
    outfile = os.path.join(output_dir, f'{code}.csv')
    subset.to_csv(outfile, index=False)
    print(f'Saved {len(subset)} rows to {outfile}')

Saved 1699343 rows to TED_by_relevant_country/PL.csv
Saved 1357576 rows to TED_by_relevant_country/FR.csv
Saved 727032 rows to TED_by_relevant_country/DE.csv
Saved 681648 rows to TED_by_relevant_country/RO.csv
Saved 422430 rows to TED_by_relevant_country/UK.csv
Saved 353252 rows to TED_by_relevant_country/ES.csv
Saved 279688 rows to TED_by_relevant_country/IT.csv
Saved 268180 rows to TED_by_relevant_country/CZ.csv
Saved 258579 rows to TED_by_relevant_country/SI.csv
Saved 203802 rows to TED_by_relevant_country/BG.csv
Saved 192964 rows to TED_by_relevant_country/LT.csv
Saved 181979 rows to TED_by_relevant_country/SE.csv
Saved 169273 rows to TED_by_relevant_country/LV.csv
Saved 121541 rows to TED_by_relevant_country/NL.csv
Saved 108124 rows to TED_by_relevant_country/HU.csv
Saved 88048 rows to TED_by_relevant_country/BE.csv
Saved 87800 rows to TED_by_relevant_country/HR.csv
Saved 84018 rows to TED_by_relevant_country/DK.csv
Saved 83643 rows to TED_by_relevant_country/GR.csv
Saved 83226 ro

In [21]:
pol = pd.read_csv('/Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/code/TED_by_relevant_country/PL.csv')

/var/folders/r2/9vnxdxyd26380tms_8k538b40000gn/T/ipykernel_49711/2595112727.py:1: DtypeWarning: Columns (8,10,14,16,17,18,20,23,24,30,33,46,52,55,57,62) have mixed types. Specify dtype option on import or set low_memory=False.
  pol = pd.read_csv('/Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/code/TED_by_relevant_country/PL.csv')


In [22]:
pol

,ID_NOTICE_CAN,TED_NOTICE_URL,YEAR,ID_TYPE,DT_DISPATCH,XSD_VERSION,CANCELLED,CORRECTIONS,B_MULTIPLE_CAE,CAE_NAME,...,NUMBER_OFFERS,NUMBER_TENDERS_SME,NUMBER_TENDERS_OTHER_EU,NUMBER_TENDERS_NON_EU,NUMBER_OFFERS_ELECTR,AWARD_EST_VALUE_EURO,AWARD_VALUE_EURO,AWARD_VALUE_EURO_FIN_1,B_SUBCONTRACTED,DT_AWARD
0,201579,ted.europa.eu/udl?uri=TED:NOTICE:79-2015:TEXT:...,2015,3,30-DEC-14,R208.S2,0,0,NaN,Urząd Morski w Słupsku,...,2.0,NaN,NaN,NaN,NaN,5855499.97,17905197.01,17905197.01,Y,30-DEC-14
1,2015361,ted.europa.eu/udl?uri=TED:NOTICE:361-2015:TEXT...,2015,3,30-DEC-14,R208.S2,0,0,NaN,Wojewódzki Specjalistyczny Szpital im. M. Piro...,...,1.0,NaN,NaN,NaN,NaN,200186.42,195137.54,195137.54,N,23-DEC-14
2,2015361,ted.europa.eu/udl?uri=TED:NOTICE:361-2015:TEXT...,2015,3,30-DEC-14,R208.S2,0,0,NaN,Wojewódzki Specjalistyczny Szpital im. M. Piro...,...,1.0,NaN,NaN,NaN,NaN,70027.01,64092.64,64092.64,N,23-DEC-14
3,2015368,ted.europa.eu/udl?uri=TED:NOTICE:368-2015:TEXT...,2015,3,30-DEC-14,R208.S2,0,0,NaN,Urząd Gminy Kłodzko,...,1.0,NaN,NaN,NaN,NaN,288255.04,246791.12,246791.12,N,22-DEC-14
4,2015374,ted.europa.eu/udl?uri=TED:NOTICE:374-2015:TEXT...,2015,3,30-DEC-14,R208.S2,0,0,NaN,Narodowy Bank Polski,...,2.0,NaN,NaN,NaN,NaN,3696806.36,3430804.86,3430804.86,Y,19-DEC-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699338,2018578400,ted.europa.eu/udl?uri=TED:NOTICE:578400-2018:T...,2018,3,28/12/18,R208.S4,0,0,NaN,Gmina Poronin,...,1.0,NaN,NaN,NaN,NaN,496304.12,663506.98,663506.98,N,27/12/18
1699339,2018578403,ted.europa.eu/udl?uri=TED:NOTICE:578403-2018:T...,2018,3,28/12/18,R208.S4,0,0,NaN,45 Wojskowy Oddział Gospodarczy w Wędrzynie,...,2.0,NaN,NaN,NaN,0.0,393476.48,317951.43,317951.43,N,19/12/18
1699340,2018578406,ted.europa.eu/udl?uri=TED:NOTICE:578406-2018:T...,2018,3,28/12/18,R208.S4,0,0,NaN,Wójt Gminy Chełmiec,...,3.0,NaN,NaN,NaN,0.0,5279831.05,429017.42,429017.42,N,03/12/18
1699341,2018578409,ted.europa.eu/udl?uri=TED:NOTICE:578409-2018:T...,2018,18,28/12/18,R208.S4,0,0,NaN,32 Baza Lotnictwa Taktycznego w Łasku,...,1.0,NaN,NaN,NaN,0.0,723692.35,799556.79,799556.79,Y,24/12/18


In [ ]:
import pandas as pd
import glob
import os

# 1. Define the folder containing your CSV files
folder_path = '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name'

# 2. List and print all CSV files found
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
print("Found files:", csv_files)

# 3. Read and concatenate if any files exist
if csv_files:
    df_list = [pd.read_csv(f) for f in csv_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    display(combined_df.head())
    output_path = os.path.join(folder_path, 'combined.csv')
    combined_df.to_csv(output_path, index=False)
    print(f"Combined {len(csv_files)} files into {output_path}")
else:
    print("No CSV files found. Check your folder_path!")

Found files: ['/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name45.csv', '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name44.csv', '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name8.csv', '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name46.csv', '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name47.csv', '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name9.csv', '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name43.csv', '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name42.csv', '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/BvD_ID_and_Name40.csv', '/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data

: 

In [1]:
import glob
import pandas as pd
import os
from tqdm import tqdm

# -- Configuration ----------------------------------------------------------
folder_path = '/Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/code/orbis_processed'    # ← adjust this!
output_path = os.path.join(folder_path, 'combined_chunked.csv')
chunk_size = 100_000                        # rows per chunk

# -- Find CSV files ---------------------------------------------------------
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
if not csv_files:
    print(f"No CSV files found in {folder_path}")
else:
    print(f"Combining {len(csv_files)} files into {output_path}...")

    header_written = False
    with open(output_path, 'w', newline='') as fout:
        # Outer bar for files
        for file_path in tqdm(csv_files, desc="Files", unit="file"):
            # Inner bar for chunks of each file
            reader = pd.read_csv(file_path, chunksize=chunk_size)
            for chunk in tqdm(reader,
                              desc=f"Chunks in {os.path.basename(file_path)}",
                              unit="chunk",
                              leave=False):
                chunk.to_csv(
                    fout,
                    index=False,
                    header=not header_written  # write header only on the very first chunk
                )
                header_written = True

    print("Done!")

Combining 47 files into /Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/code/orbis_processed/combined_chunked.csv...


Files: 100%|██████████| 47/47 [14:43<00:00, 18.79s/file]

Done!


In [3]:
all_names = pd.read_csv("/Users/wiktorrajca/Desktop/Research/URAP_Fedyk/data/Orbis_Data/BvD_ID_and_Name/combined_chunked.csv")

: 

In [6]:
uk = pd.read_csv('/Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/code/TED_by_relevant_country/UK.csv')

/var/folders/r2/9vnxdxyd26380tms_8k538b40000gn/T/ipykernel_6233/2746756887.py:1: DtypeWarning: Columns (8,10,14,16,17,18,20,23,24,33,46,55,57,62) have mixed types. Specify dtype option on import or set low_memory=False.
  uk = pd.read_csv('/Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/code/TED_by_relevant_country/UK.csv')


In [8]:
md = pd.read_csv('/Users/wiktorrajca/Documents/GitHub/Data-Science-Honors-Thesis/code/TED_by_relevant_country/MD.csv')

In [9]:
md

,ID_NOTICE_CAN,TED_NOTICE_URL,YEAR,ID_TYPE,DT_DISPATCH,XSD_VERSION,CANCELLED,CORRECTIONS,B_MULTIPLE_CAE,CAE_NAME,...,NUMBER_OFFERS,NUMBER_TENDERS_SME,NUMBER_TENDERS_OTHER_EU,NUMBER_TENDERS_NON_EU,NUMBER_OFFERS_ELECTR,AWARD_EST_VALUE_EURO,AWARD_VALUE_EURO,AWARD_VALUE_EURO_FIN_1,B_SUBCONTRACTED,DT_AWARD
0,2015323439,ted.europa.eu/udl?uri=TED:NOTICE:323439-2015:T...,2015,3,10-SEP-15,R208.S2,0,0,NaN,Posch & Partner GmbH on behalf of the Municipa...,...,5.0,NaN,NaN,NaN,NaN,160000.00,106648.70,106648.70,N,30-JUN-15
1,201464451,ted.europa.eu/udl?uri=TED:NOTICE:64451-2014:TE...,2014,3,21-FEB-14,R208.S2,0,0,NaN,Pramacom-HT; spol. s r.o.,...,2.0,NaN,NaN,NaN,NaN,50842.53,51871.28,51871.28,N,07-FEB-14
2,2014160115,ted.europa.eu/udl?uri=TED:NOTICE:160115-2014:T...,2014,3,09-MAY-14,R208.S2,0,0,NaN,Fundacja Rozwoju Systemu Edukacji,...,1.0,NaN,NaN,NaN,NaN,6638.57,8960.00,8960.00,N,24-FEB-14
3,2014160115,ted.europa.eu/udl?uri=TED:NOTICE:160115-2014:T...,2014,3,09-MAY-14,R208.S2,0,0,NaN,Fundacja Rozwoju Systemu Edukacji,...,1.0,NaN,NaN,NaN,NaN,6638.57,8960.00,8960.00,N,24-FEB-14
4,2016153871,ted.europa.eu/udl?uri=TED:NOTICE:153871-2016:T...,2016,3,02-MAY-16,R208.S2,0,0,NaN,Posch and Partner GmbH on behalf of the Munici...,...,4.0,NaN,NaN,NaN,0.0,160000.00,106648.70,106648.70,N,30-JUL-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2022592377,ted.europa.eu/udl?uri=TED:NOTICE:592377-2022:T...,2022,6,21/10/22,R209.S5,0,0,N,COMPANIA MUNICIPALA TERMOENERGETICA S.A.,...,4.0,4.0,0.0,1.0,4.0,265344.40,265344.40,265344.40,N,13/10/22
97,2022592377,ted.europa.eu/udl?uri=TED:NOTICE:592377-2022:T...,2022,6,21/10/22,R209.S5,0,0,N,COMPANIA MUNICIPALA TERMOENERGETICA S.A.,...,4.0,4.0,0.0,1.0,4.0,751526.03,751526.03,751526.03,N,13/10/22
98,2022668918,ted.europa.eu/udl?uri=TED:NOTICE:668918-2022:T...,2022,6,28/11/22,R209.S5,0,0,N,COMPANIA MUNICIPALA TERMOENERGETICA S.A.,...,4.0,4.0,0.0,1.0,4.0,248089.39,248089.39,248089.39,N,24/11/22
99,2022668918,ted.europa.eu/udl?uri=TED:NOTICE:668918-2022:T...,2022,6,28/11/22,R209.S5,0,0,N,COMPANIA MUNICIPALA TERMOENERGETICA S.A.,...,3.0,3.0,0.0,1.0,3.0,194836.21,194836.21,194836.21,N,24/11/22


# CREATING MERGED DATA FOR EACH COUNTRY